In [48]:
import numpy as np
import pandas as pd

#Carga de datos
progNueva = pd.read_csv('progNueva.csv')
progTotal = pd.read_csv('progTotal.csv')

#prog = pd.read_csv('progNueva.csv')
progNueva['fechaVenta'] = pd.to_datetime(progNueva['fechaVenta'])
progTotal['fechaVenta'] = pd.to_datetime(progTotal['fechaVenta'])
progTotal['fechaProg'] = pd.to_datetime(progTotal['fechaProg'])

progNueva['fechaProg'] = progNueva['fechaVenta'] + pd.to_timedelta((progNueva['Semana_prog']-1), unit='w')
progTotal

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,9,0,9,2018-01-02,6,2018-01-02
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,2018-01-02,36,2018-01-02
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,2018-01-02,36,2018-01-02
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,2018-01-02,1,2018-01-02
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,2018-01-03,1,2018-01-03
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,2018-01-03,1,2018-01-03


In [49]:
# extraer coincidencias entre las programaciones, sumar Q´s, sumar semana_prog, conservar fechaVenta original, semana_prog, fechaProg
innerProg = pd.merge(progTotal, progNueva, how='inner', left_on=['Nombre_Clinica','dni','ttmto_homol'], right_on=['Nombre_Clinica','dni','ttmto_homol'])
innerProg['Cantidad'] = innerProg['Cantidad_x'] + innerProg['Cantidad_y']
innerProg['Cant_prod'] = innerProg['Cant_prod_x'] + innerProg['Cant_prod_y']
innerProg['Cant_pend'] = innerProg['Cant_pend_x'] + innerProg['Cant_pend_y']
innerProg = innerProg[['Nombre_Clinica','Paciente_x','dni','ttmto_homol','Cantidad','Cant_prod','Cant_pend','fechaVenta_x','Semana_prog_x','fechaProg_x']]
innerProg = innerProg.rename(index=str, columns={"Paciente_x":"Paciente","fechaVenta_x":"fechaVenta","Semana_prog_x":"Semana_prog",'fechaProg_x':'fechaProg'})
innerProg

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,12,0,12,2018-01-02,6,2018-01-02


In [51]:
leftProg = pd.merge(progTotal, progNueva, how='left', left_on=['Nombre_Clinica','dni','ttmto_homol'], right_on=['Nombre_Clinica','dni','ttmto_homol'])
leftProg = leftProg[leftProg['Paciente_y'].isnull()]
leftProg = leftProg[['Nombre_Clinica','Paciente_x','dni','ttmto_homol','Cantidad_x','Cant_prod_x','Cant_pend_x','fechaVenta_x','Semana_prog_x','fechaProg_x']]
leftProg = leftProg.rename(index=str, columns={"Paciente_x":"Paciente","fechaVenta_x":"fechaVenta","Semana_prog_x":"Semana_prog",'fechaProg_x':'fechaProg','Cantidad_x':'Cantidad','Cant_prod_x':'Cant_prod','Cant_pend_x':'Cant_pend'})
leftProg

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,2018-01-02,36,2018-01-02
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,2018-01-02,36,2018-01-02
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,2018-01-02,1,2018-01-02
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,2018-01-03,1,2018-01-03
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,2018-01-03,1,2018-01-03


In [53]:
progTotal = leftProg.append(innerProg)

progTotal = progTotal.groupby(['Nombre_Clinica','Paciente','dni','ttmto_homol'], as_index = False).sum()
progTotal['PQR'] = 0

print("Suma de Programaciones:")
progTotal

Suma de Programaciones:


,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,Semana_prog,PQR
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,12,0,12,6,0
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,36,0
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,36,0
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,1,0
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,1,0
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,1,0


In [50]:
prod_df['Cant'] = (prod_df['Importe Producido'] + prod_df['Importe Descuento'])/prod_df['Importe Tratamiento']
prodResumen = prod_df.groupby(['Clínica','Paciente','DNI','ttmto_homol'], as_index = False).sum()
prodResumen = prodResumen.rename(index=str, columns={"Clínica": "Nombre_Clinica"})
prodResumen

,Nombre_Clinica,Paciente,DNI,ttmto_homol,Nº Historia,Presupuesto,Duración,Importe Tratamiento,Importe Descuento,Importe Producido,Equivalencia,temp,Cant
0,Autopista 104A Bogota,EVELYN DAZA,111.0,MENSUALIDAD ORTODONCIA METALICA,492.0,20.0,120.0,280000.0,28000.0,252000.0,0.0,False,4.0
1,Autopista 104A Bogota,EVELYN DAZA,111.0,RETIRO BRACKETS ARCADA,123.0,10.0,30.0,90000.0,0.0,90000.0,0.0,False,1.0
2,Autopista 104A Bogota,PEDRO GOMEZ,222.0,DETARTRAJE POR ARCADA,123.0,5.0,30.0,70000.0,7000.0,63000.0,0.0,False,1.0
3,Autopista 104A Bogota,PEDRO GOMEZ,222.0,PROFILAXIS,369.0,15.0,90.0,210000.0,21000.0,189000.0,0.0,False,3.0


In [51]:
# CONCIDENCIAS ENTRE VENTAS Y PROD
inner_df = pd.merge(progTotal, prodResumen, how='inner', left_on=['Nombre_Clinica','Paciente','dni','ttmto_homol'], right_on=['Nombre_Clinica','Paciente','DNI','ttmto_homol'])

# ACTUALIZACION DE CANTIDADES PENDIENTES
inner_df['Cant_Pend'] = inner_df['Cantidad'] - inner_df['Cant']

inner_df = inner_df[['Nombre_Clinica','Paciente','dni','ttmto_homol','Cant_Pend','Semana_prog']] #el tel del paciente no está en los DFs. Solicitar

# ACTUALIZACION DE PROGRAMACION DE ORTODONCIA METALICA
boolMask = inner_df['ttmto_homol']=='MENSUALIDAD ORTODONCIA METALICA'
#inner_df.loc[boolMask]['Cant_Pend']-1
inner_df.loc[boolMask,'Semana_prog'] = inner_df.loc[boolMask,'Semana_prog']+4 #aqui no es asi: [semana_produccion] +4
inner_df

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,Semana_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,5.0,10
1,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,1.0,1
2,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,1.0,1
3,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,-1.0,1


In [32]:
# extraer procedimientos de VENTAS que no cruzaron con PROD
df1 = pd.merge(progTotal, prodResumen, how='left', left_on=['Nombre_Clinica','dni','ttmto_homol'], right_on=['Nombre_Clinica','DNI','ttmto_homol'])
df1 = df1[df1['Cant'].isnull()] #probar con la func isnan()
df1

,Nombre_Clinica,Paciente_x,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,Semana_prog,Paciente_y,DNI,Nº Historia,Presupuesto,Duración,Importe Tratamiento,Importe Descuento,Importe Producido,Equivalencia,temp,Cant
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,2,0,2,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,4,0,4,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#acomodar procedimientos que no cruzaron de la misma manera
df1 = df1.rename(index=str, columns={"Cantidad": "Cant_Pend"})
df1 = df1[['Nombre_Clinica','Paciente_x','dni','ttmto_homol','Cant_Pend','Semana_prog']]
df1 = df1.rename(index=str, columns={"Paciente_x": "Paciente"})
df1

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,Semana_prog
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,2,36
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,4,36


In [34]:
#concatenar los procedimientos que no cruzaron con los que si cruzaron (PROGRAMA DE PRODUCCION)
pd.concat([df1, inner_df], ignore_index=True)


,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,Semana_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,2.0,36
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,4.0,36
2,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,14.0,10
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,3.0,1
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,3.0,1
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,1.0,1
